# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,18.28,65,78,6.22,PN,1724384168
1,1,qaqortoq,60.7167,-46.0333,8.01,60,96,1.86,GL,1724384244
2,2,laguna,38.4210,-121.4238,20.69,65,0,5.66,US,1724384246
3,3,ushuaia,-54.8000,-68.3000,6.81,70,75,7.72,AR,1724383968
4,4,mykhaylivka,47.2724,35.2224,22.45,64,19,2.95,UA,1724384249


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 25
min_temp = 17
max_wind = 15
max_humidity = 50

filtered_city_df = city_data_df[
    (city_data_df['Max Temp'] > min_temp) &
    (city_data_df['Max Temp'] <= max_temp) &
    (city_data_df['Wind Speed'] < max_wind) &
    (city_data_df['Humidity'] < max_humidity)
]

# Drop any rows with null values
filtered_city_df_clean = filtered_city_df.dropna()

# Display sample data
filtered_city_df_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,claudio,-20.4433,-44.7658,17.47,46,1,0.60,BR,1724384267
38,38,zavetnoye,47.1194,43.8903,24.69,27,14,7.61,RU,1724384289
59,59,susuman,62.7833,148.1667,17.95,38,26,1.30,RU,1724384316
96,96,klyuchi,52.2667,79.1667,23.15,41,98,4.70,RU,1724384360
99,99,kuanda,56.3161,116.0806,24.48,39,99,2.29,RU,1724384363


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_df_clean[['City', 'Lat', 'Lng', 'Country', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ' '

# Display sample data
hotel_df.head()

,City,Lat,Lng,Country,Humidity,Hotel Name
19,claudio,-20.4433,-44.7658,BR,46,
38,zavetnoye,47.1194,43.8903,RU,27,
59,susuman,62.7833,148.1667,RU,38,
96,klyuchi,52.2667,79.1667,RU,41,
99,kuanda,56.3161,116.0806,RU,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
        'categories': 'accommodation.hotel',
        'apiKey': geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
claudio - nearest hotel: No hotel found
zavetnoye - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
kuanda - nearest hotel: No hotel found
ust-nera - nearest hotel: гостиница "Солнечная"
hovd - nearest hotel: Khovd hotel
port lincoln - nearest hotel: Boston Hotel
olyokminsk - nearest hotel: Колос
miles city - nearest hotel: Historic Olive Hotel
maun - nearest hotel: Center Lodge Conference Center
vernon - nearest hotel: Okanagan Royal Park Inn
oliver - nearest hotel: Coast Oliver Hotel
fort st. john - nearest hotel: Fort St. John Motor Inn
pendleton - nearest hotel: Howard Johnson Pendleton
kabalo - nearest hotel: No hotel found
nyurba - nearest hotel: Марина
glendive - nearest hotel: La Quinta Inn & Suites Glendive
krasnyy chikoy - nearest hotel: Гостиница ЗАО "Слюдянка"
borzya - nearest hotel: Оазис
tahe - nearest hotel: No hotel found
rio preto da eva - nearest hotel: Hotel Santo Antonio
cianorte - n

,City,Lat,Lng,Country,Humidity,Hotel Name
19,claudio,-20.4433,-44.7658,BR,46,No hotel found
38,zavetnoye,47.1194,43.8903,RU,27,No hotel found
59,susuman,62.7833,148.1667,RU,38,No hotel found
96,klyuchi,52.2667,79.1667,RU,41,No hotel found
99,kuanda,56.3161,116.0806,RU,39,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    hover_cols = ["Hotel Name", "Country"],
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)